In [1]:
from fastapi import FastAPI, Request, status
from fastapi.responses import Response
import nest_asyncio
nest_asyncio.apply()
import uvicorn
from pydantic import BaseModel
from dotenv import load_dotenv
load_dotenv()
import openai
import os
import sys
from pathlib import Path
from openai import OpenAI
import ast
import json
from fastapi.middleware.cors import CORSMiddleware

In [2]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not len(OPENAI_API_KEY):
    print("Set OPENAI_API_KEY")


In [28]:
class Speech(BaseModel):
    excerpt:str
    previous_suggestion:str

app = FastAPI()

origins = [
    "http://localhost:4321",
]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials = True,
    allow_methods=["*"],
    allow_headers=["*"]
)




In [26]:
def generate_speech(excerpt, previous_suggestion):
    prompt = [{"role":"system","content":'''Your job is to continue my speech in 1 sentence. Your output should be a json with only one key, which is next_sentence. Refer to your previous suggestion. 
               If user is following your previous suggestion. The value of next_sentence should be the part of the previous suggestion that is not yet included in the excerpt plus a new sentence. Else,
               the value of next_sentence should just be a new sentence.'''},
              {"role":"user","content":f"Previous suggestion: {previous_suggestion}"},
              {"role":"user","content":f"Speech so far: {excerpt}"}]
    return prompt

In [29]:
@app.post("/generate_speech",status_code=status.HTTP_202_ACCEPTED)
def continue_speech(speech:Speech):
    try:
        client = OpenAI()
        response = client.chat.completions.create(
            model = "gpt-3.5-turbo-0125",
            response_format={"type":"json_object"},
            temperature=0.7,
            messages=generate_speech(excerpt=speech.excerpt,previous_suggestion=speech.previous_suggestion)
        )
        return response.choices[0].message.content
    except Exception as e:
        print("ERROR!")
        return {"Error":e}

uvicorn.run(app, host="192.168.0.23",port=8080)

INFO:     Started server process [18564]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://192.168.0.23:8080 (Press CTRL+C to quit)


INFO:     192.168.0.23:53329 - "OPTIONS /generate_speech HTTP/1.1" 200 OK
INFO:     192.168.0.23:53329 - "POST /generate_speech HTTP/1.1" 422 Unprocessable Entity
INFO:     192.168.0.23:53330 - "POST /generate_speech HTTP/1.1" 202 Accepted
INFO:     192.168.0.23:53340 - "POST /generate_speech HTTP/1.1" 202 Accepted


INFO:     Shutting down
INFO:     Waiting for connections to close. (CTRL+C to force quit)


INFO:     192.168.0.23:53343 - "POST /generate_speech HTTP/1.1" 202 Accepted


INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [18564]
